In [1]:
import pandas as pd
import numpy as np
import os

INTERIM_DIR = '../data/interim/'
PROCESSED_DIR = '../data/processed/'
ENRICHED_FILE = os.path.join(INTERIM_DIR, 'sales_data_enriched.csv')
CLEAN_FILE = os.path.join(PROCESSED_DIR, 'final_cleaned_data.csv')

In [2]:
dtype_spec = {
    'sale_price': 'object',
    'class': 'str',
    'pin': 'str',
    'pin10': 'str'
}
df = pd.read_csv(ENRICHED_FILE, dtype=dtype_spec)
df.shape

/var/folders/fk/9chv4jr11ln50p4dhsxntpcw0000gn/T/ipykernel_45845/3334969353.py:7: DtypeWarning: Columns (8,10,15,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENRICHED_FILE, dtype=dtype_spec)


(1755174, 26)

In [3]:
# Convert sale_price to numeric (handles '$' and commas)
df['sale_price'] = df['sale_price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

In [4]:
# Drop rows where critical enriched features are missing (should be few after initial enrichment)
df.dropna(subset=['sale_price', 'lon', 'lat', 'min_distance_meters'], inplace=True)
print(f"Shape after critical null drops: {df.shape}")

Shape after critical null drops: (1755174, 26)


In [5]:
print(df['class'].unique())
RESIDENTIAL_CLASSES = ['202', '203', '204', '205', '206', '207', '208', '209']
df['class'] = df['class'].astype(str).str[:3]
df_filtered = df[df['class'].isin(RESIDENTIAL_CLASSES)].copy()
df_filtered.shape

['278' '299' '203' '100' '205' '590' '218' '204' '592' '201' '211' '399'
 '207' '517' '315' '202' '580' '234' '593' '390' 'RR' '523' '295' '528'
 'EX' '212' '597' '321' '241' '314' '239' '210' '397' '221' '591' '319'
 '206' '522' '220' '290' '190' '527' '391' '318' '516' '530' '294' '599'
 '396' '490' '663' '297' '529' '209' '533' '320' '670' '535' '208' '219'
 '288' '213' '531' '897' '880' '589' '831' '581' '236' '532' '817' '224'
 '638' '893' '200' '435' '313' '583' '797' '550' '693' '587' '301' '915'
 'OA3' '526' '914' '991' '417' '492' '765' '501' '654' '491' '997' '790'
 '497' '679' '225' '500' '921' '677' '990' '913' '890' '689' '918' '493'
 '420' '680' '891' '730' '791' '499' '421' '887' '919' '717' '823' '883'
 '996' '428' '899' '827' '668' '833' '828' '889' '483' '729' '892' '801'
 '792' '830' '673' '663A' '767' '300' '663B' '670B' 'OA1' '817A' '817B'
 '717A' '822' '829' '893A' '893B' '729A']


(529212, 26)

In [6]:
df_filtered = df_filtered[
    (df_filtered['is_multisale'] == False) &
    (df_filtered['sale_filter_less_than_10k'] == False) &
    (df_filtered['sale_filter_deed_type'] == False)
]
df_filtered.shape

(440343, 26)

In [7]:
df_filtered['sale_date'] = pd.to_datetime(df_filtered['sale_date'], errors='coerce')
RECENT_YEAR = 2018
df_filtered = df_filtered[df_filtered['sale_date'].dt.year >= RECENT_YEAR]
df_filtered.shape

(124917, 26)

In [8]:
lower_bound = df_filtered['sale_price'].quantile(0.01)
upper_bound = df_filtered['sale_price'].quantile(0.99)
df_final = df_filtered[
    (df_filtered['sale_price'] >= lower_bound) & 
    (df_filtered['sale_price'] <= upper_bound)
].copy()
df_final.describe()

,year,township_code,nbhd,sale_date,sale_price,num_parcels_sale,row_id,lon,lat,min_distance_meters
count,122544.000000,122544.000000,122544.000000,122544,1.225440e+05,122544.0,1.225440e+05,122544.000000,122544.000000,122544.000000
mean,2021.489498,47.399693,47533.384213,2021-12-25 18:30:13.396004864,2.882095e+05,1.0,3.425131e+07,-87.728609,41.783629,7461.687594
min,2018.000000,10.000000,10011.000000,2018-01-01 00:00:00,2.500000e+04,1.0,7.087464e+06,-88.027405,41.469928,29.502009
25%,2020.000000,27.000000,27020.000000,2020-04-29 00:00:00,1.550000e+05,1.0,7.338743e+06,-87.794373,41.686748,2078.036263
50%,2021.000000,39.000000,39080.000000,2021-12-06 00:00:00,2.465000e+05,1.0,7.600606e+06,-87.728363,41.778469,4642.443845
75%,2023.000000,72.000000,72030.000000,2023-09-19 00:00:00,3.505000e+05,1.0,9.651593e+07,-87.664229,41.907803,10293.319875
max,2025.000000,77.000000,77170.000000,2025-09-29 00:00:00,1.545000e+06,1.0,9.848484e+07,-87.524891,42.065343,34227.066903
std,2.100147,23.873829,23924.266033,NaN,2.103589e+05,0.0,4.112320e+07,0.092687,0.144395,7358.454198


In [9]:
df_final.to_csv(CLEAN_FILE, index=False)